In [ ]:
import praw
import pandas as pd
from praw.models import MoreComments

# #URL to access the app needed to scrape the data off the Berkeley subreddit
# # https://www.reddit.com/prefs/apps

#This is the tutorial I used to set up the web scraping using PRAW
#https://www.geeksforgeeks.org/python/scraping-reddit-using-python/

# #PLAN
# #Scrape the data off the Berkeley subreddit about consulting clubs
# #Organize that data using the pandas library
# #Create chatbot that utilizes natural language processing that will give users feedback
# #about each consulting club here at Berkeley. 

reddit_read_only = praw.Reddit(client_id = "QlBfNfxQ3e_MGP9RkaOQig",
                               client_secret = "SpLjOwYdQPU4z1wqcXBjVl_7DnUIZg",
                               user_agent = "Berkeley_Consulting")

subreddit = reddit_read_only.subreddit("berkeley")

# df1 = pd.DataFrame({
#     'post_id': ['p1', 'p2', 'p3'],
#     'title': ['Title A', 'Title B', 'Title C'],
#     'author': ['user1', 'user2', 'user3']
# })

# df2 = pd.DataFrame({
#     'comment_id': ['c1', 'c2', 'c3', 'c4', 'c5'],
#     'post_id': ['p1', 'p1', 'p2', 'p2', 'p4'],  # Note: p4 doesn't exist in df1
#     'comment': ['Great post!', 'Thanks!', 'Interesting', 'I agree', 'Nice!']
# })
# outer_merged = df1.merge(df2, on='post_id', how='outer')
# print(outer_merged)

In [2]:
consulting_posts = subreddit.search('consulting')

posts_dict = {'Title': [], 'Post Text': [], 'ID': [], 'Score': [], 'Total Comments': [], 'Post URL': []}
comments_dict = {'Comment': [], 'Score': []}

In [3]:
for post in consulting_posts:
    #The title of the post
    posts_dict['Title'].append(post.title)
    #The text inside of the post
    posts_dict['Post Text'].append(post.selftext)
    #Unique ID of each post
    posts_dict['ID'].append(post.id)
    #The scoure of a post
    posts_dict['Score'].append(post.score)
    #Total number of comments inside the post
    posts_dict['Total Comments'].append(post.num_comments)
    #URL of each post
    posts_dict['Post URL'].append(post.url)

consulting_club_posts = pd.DataFrame(posts_dict)
# print(consulting_club_posts)
consulting_club_posts

,Title,Post Text,ID,Score,Total Comments,Post URL
0,Asian monoculture in consulting clubs,Over the past month on this subreddit there ha...,16o5u8z,267,112,https://www.reddit.com/r/berkeley/comments/16o...
1,What are consulting clubs actually looking for?,I am an incoming freshman at Haas next year an...,1kvi9jt,43,55,https://www.reddit.com/r/berkeley/comments/1kv...
2,Tech PM blocks all “.berkeley.edu” e-mails bc ...,Consulting clubs making us look bad smh,19d8ceg,563,43,https://i.redd.it/em6d28vzh2ec1.jpeg
3,Rejected from consulting club after having bee...,titles kinda self explanatory lol. didn’t even...,1na99k8,76,22,https://www.reddit.com/r/berkeley/comments/1na...
4,Why are some of these Business/Consulting Club...,My friend and I were enjoying dinner in the so...,16jsy19,367,55,https://www.reddit.com/r/berkeley/comments/16j...
...,...,...,...,...,...,...
95,another bear rejected by consulting clubs,I know it's just a club but man I wanna cry i ...,piudil,31,7,https://www.reddit.com/r/berkeley/comments/piu...
96,There are 150-odd chapters of Chinese Students...,,69fs7e,51,22,https://www.nytimes.com/2017/05/04/us/chinese-...
97,NEW! - Stress Management Consultations,Stress Management Consultations at the UHS Car...,zdf1h4,7,0,https://www.reddit.com/r/berkeley/comments/zdf...
98,Made it to a Consulting Club’s Final Round Int...,I’m not happy about getting rejected but I did...,sjnb6y,3,6,https://www.reddit.com/r/berkeley/comments/sjn...


In [ ]:
#Need to figure out a way to get the URL of each and every post
for i in posts_dict['Post URL']:
    if 'comments' in i:
        submission = reddit_read_only.submission(url = i)
    else:
        continue
    for comment in submission.comments:
        if len(comment.body) < 20:
            continue
        comments_dict['Comment'].append(comment.body)
        comments_dict['Score'].append(comment.score)

# print(post_comments[0]) #This only prints one comment right now for one URL. Need to get as many comments as possible for one given URL.

comments_df = pd.DataFrame(comments_dict)
comments_df

#Generate the CSV file that contains all the posts about Berkeley clubs
# comments_df.to_csv('my_dataframe.csv', index=False) # index=False prevents writing the DataFrame index as a column

#They are not marked in any way so they are just random pieces of information
#Need to find a way to associate either the post title or the post URL with the post comments

In [6]:
# Creating weights for each embedding based on score values, plugging them into a tuned sigmoid function
# 0-200 has factor ~1, 200-500 has factor ~1.3, 500-700 has factor ~1.6, 700+ has factor ~1.8
# Doesn't function well, need to include comment score scaling as well if implemented

# consulting_club_posts['Score Factor'] = consulting_club_posts['Score'].apply(lambda x: 1 + 1/(1+20*pow(2, -x/100)))

# consulting_club_posts['Weighted embedding'] = consulting_club_posts['embedding'].combine(consulting_club_posts['Score Factor'], lambda lst, weight: [weight * x for x in lst])
# print(consulting_club_posts['Weighted embedding'], consulting_club_posts['Score'])

In [7]:
def cosine_similarity(a, b):
    dot_prod = sum([x * y for x, y in zip(a, b)])
    mag_a = pow(sum([pow(x, 2) for x in a]), 0.5)
    mag_b = pow(sum([pow(y, 2) for y in b]), 0.5)
    return mag_a * mag_b and (dot_prod) / (mag_a * mag_b)   # add mag_b to dot_prod for weightage

In [8]:
def retrieve_data(query, n=5):
    query_embed = ollama.embed(model=embed_model, input=query)['embeddings']
    similarities = consulting_club_posts['embedding'].apply(lambda x: cosine_similarity(query_embed[0], x))     # Change to 'weighted embedding' for weightage
    pd.concat([similarities, comments_df['embedding'].apply(lambda x: cosine_similarity(query_embed[0], x))])   
    return similarities.nlargest(n)

In [9]:
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve_data(input_query)


instruction_prompt = f'''You are a helpful chatbot aimed to help UC Berkeley students learn about and choose clubs to join.
Use only the following pieces of context to answer the question. Don't make up any new information:
{'\n'.join([f' - {consulting_club_posts.loc[i, 'Post Text']}' for i in retrieved_knowledge.index])}
'''

print(instruction_prompt)

IndexError: list index out of range

In [ ]:
stream = ollama.chat(
  model=lang_model,
  messages=[
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)

# print the response from the chatbot in real-time
print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)


Chatbot response:
Here's some general information about the tech-specific consulting clubs at UC Berkeley:

* **Berkeley Consulting**: A well-established club that provides consulting services to clients. While it's considered prestigious, getting into it is competitive, and you'll likely need a strong understanding of business and operations.
* **Voyager Consulting**: The "vanguard" of consulting clubs on campus, known for their innovative approaches and high demand among top companies.
* **Venture Strategy Solutions (VSS)**: A highly respected club that focuses on venture capital analysis, strategy, and team management. They're often considered one of the most prestigious consulting clubs at UC Berkeley.

To increase your chances of getting into these clubs, consider the following:

1. **Genuinely impressive projects**: Many successful consultants start with smaller projects or initiatives that demonstrate their capabilities and values.
2. **Strong internship experience**: If you're 